```{.dockerfile include="../_version.md"}
```

Cette partie illustre les différentes approches pour générer des images docker pour des applications Java.

In [1]:
#| output: false
#| echo: false
docker system prune --all --force

Total reclaimed space: 0B


## A partir d'une image de maven

Le plus simple est de s'appuyer sur une image de maven, d'y copier les sources et de donner comme commande le résutat de compilation.

```{.dockerfile include="sample-java/helloworld/Dockerfile.01.mavenimage" code-line-numbers="true"}
```

In [2]:
(cd sample-java/helloworld && \
docker image build --quiet --tag javahello:mavenimage --file Dockerfile.01.mavenimage . )

cd: no such file or directory: sample-java/helloworld


In [3]:
docker image ls|grep javahello|grep "mavenimage "|tr -s ' '

In [4]:
docker run --rm javahello:mavenimage

Unable to find image 'javahello:mavenimage' locally


docker: Error response from daemon: pull access denied for javahello, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.


See 'docker run --help'.


## Multistage

Il est plus efficace de s'appuyer sur la fabrication multistage d'une image docker pour séparer la chaîne de compilation de l'image d'exécution.
On utilise alors comme image finale une base de JRE (donc plus légère) dans laquelle uniquement le résultat de la compilation est copié (attention, il doit inclure les dépendances cf. uberjar). 

```{.dockerfile include="sample-java/helloworld/Dockerfile.02.mavenimagestage" code-line-numbers="true"}
```

In [5]:
(cd sample-java/helloworld && \
docker image build --quiet --tag javahello:mavenimagestage --file Dockerfile.02.mavenimagestage . )

cd: no such file or directory: sample-java/helloworld


In [6]:
docker image ls|grep javahello|grep mavenimagestage|tr -s ' '

In [7]:
docker run --rm javahello:mavenimagestage

Unable to find image 'javahello:mavenimagestage' locally


docker: Error response from daemon: pull access denied for javahello, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.


See 'docker run --help'.


## Cache docker

Les versions récentes de Docker proposent un mécanisme de cache qui permet de préciser explictement un dossier à mettre en cache lors du build d'une image. Pour Java et Maven, on peut ainsi mettre en cache `~/.m2` pour éviter de retélécharger systématiquement.

```{.dockerfile include="sample-java/helloworld/Dockerfile.03.cache" code-line-numbers="true"}
```

In [8]:
(cd sample-java/helloworld && \
docker image build --quiet --tag javahello:cache --file Dockerfile.03.cache . )

cd: no such file or directory: sample-java/helloworld


In [9]:
docker image ls|grep javahello|grep cache|tr -s ' '

In [10]:
docker run --rm javahello:cache

Unable to find image 'javahello:cache' locally


docker: Error response from daemon: pull access denied for javahello, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.


See 'docker run --help'.


## Maven wrapper

Il est aussi possible de s'appuyer simplement sur l'image d'un JDK et d'utiliser un [wrapper maven](https://maven.apache.org/wrapper/). Dans ce cas, la bonne version de maven sera téléchargée au moment de la compilation.

```{.dockerfile include="sample-java/helloworld/Dockerfile.04.wrapper" code-line-numbers="true"}
```

In [11]:
(cd sample-java/helloworld && \
docker image build --quiet --tag javahello:wrapper --file Dockerfile.04.wrapper . )

cd: no such file or directory: sample-java/helloworld


In [12]:
docker image ls|grep javahello|grep wrapper|tr -s ' '

In [13]:
docker run --rm javahello:wrapper

Unable to find image 'javahello:wrapper' locally


docker: Error response from daemon: pull access denied for javahello, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.


See 'docker run --help'.


## From scratch
Il est aussi possible réaliser une chaine de compilation personnalisée à partir de zéro par exemple avec [sdkman](https://sdkman.io/). 

```{.dockerfile include="sample-java/helloworld/Dockerfile.05.scratch" code-line-numbers="true"}
```

In [14]:
(cd sample-java/helloworld && \
docker image build --quiet --tag javahello:scratch --file Dockerfile.05.scratch . )

cd: no such file or directory: sample-java/helloworld


In [15]:
docker image ls|grep javahello|grep scratch|tr -s ' '

In [16]:
docker run --rm javahello:scratch

Unable to find image 'javahello:scratch' locally


docker: Error response from daemon: pull access denied for javahello, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.


See 'docker run --help'.


## Avec les modules et jlink 

La meilleure solution est d'utiliser les modules de Java et jlink pour produire une livraison minimale du JRE et des dépendances pour le projet. 

**TODO**

## Comparaison des images

In [17]:
docker image ls|grep javahello